In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import Select
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [6]:
driver = webdriver.Edge()
driver.get('https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/TarifasCREIndustria/Tarifas/DemandaIndustrialSub.aspx')

In [ ]:
year_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha_ddAnio'))
month_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha2_ddMes'))
state_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddEstado'))
#municip_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
#division_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))

In [ ]:
#year_options = [option.text for option in year_selection.options]
#month_options = [option.text for option in month_selection.options]
#month_options = month_options[1:]
#state_options = [option.text for option in state_selection.options]
#state_options = state_options[1:]

In [4]:
year_options = ['2022', '2021', '2020', '2019', '2018', '2017']
month_options = list(range(1,13))
state_options = list(range(1,33))

In [31]:
month_options

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [21]:
driver.close()

In [19]:
dfs = []
driver = webdriver.Edge()
driver.get('https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/TarifasCREIndustria/Tarifas/DemandaIndustrialSub.aspx')
year_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha_ddAnio'))
year_selection.select_by_value('2022')
time.sleep(0.5)
for month in month_options:
    month_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha2_ddMes'))
    month_selection.select_by_value(str(month))
    time.sleep(0.5)
    for state in state_options:
        state_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddEstado'))
        state_selection.select_by_value(str(state))
        time.sleep(0.5)
        municip_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
        municip_options = [option.text for option in municip_selection.options]
        municip_values = [option.get_attribute("value") for option in municip_selection.options]
        municip_options = municip_options[1:]
        time.sleep(0.5)
        for index_municip in municip_values[1:]:
            #print(index_municip)
            select_mun = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
            select_mun.select_by_value(str(index_municip))

            select_div = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))
            div_options = [option.text for option in select_div.options]
            #print(div_options)
            div_values = [option.get_attribute("value") for option in select_div.options]

            for index_division in div_values[1:]:
                select_div = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))
                select_div.select_by_value(str(index_division))
                time.sleep(0.5)
                table = driver.find_element(By.CSS_SELECTOR,"table.table.table-bordered.table-striped")
                table_html = table.get_attribute('outerHTML')
                df = pd.read_html(table_html)[0]
                df['year'] = 2022
                df['month'] = month
                df['state'] = state
                df['municipality'] = index_municip
                df['division'] = index_division
                dfs.append(df)

data = pd.concat(dfs)
data.to_csv('data_2022.csv', index=False)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"table.table.table-bordered.table-striped"}
  (Session info: MicrosoftEdge=107.0.1418.42)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF686558502+9986]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF6864F5D32+1445154]
	Ordinal0 [0x00007FF6860DFC8C+654476]
	Ordinal0 [0x00007FF6861236C2+931522]
	Ordinal0 [0x00007FF686123B10+932624]
	Ordinal0 [0x00007FF68615FC17+1178647]
	Ordinal0 [0x00007FF686143BDF+1063903]
	Ordinal0 [0x00007FF686115FF4+876532]
	Ordinal0 [0x00007FF68615CF70+1167216]
	Ordinal0 [0x00007FF6861439B3+1063347]
	Ordinal0 [0x00007FF68611506A+872554]
	Ordinal0 [0x00007FF68611402E+868398]
	Ordinal0 [0x00007FF68611570F+874255]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF6863B6108+135416]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF6863A02C

In [20]:
pd.concat(dfs)

,Tarifa,Descripción,Int. Horario,Cargo,Unidades,ENE-22,year,month,state,municipality,division
0,DIST,Demanda industrial en subtransmisión,-,Fijo,$/mes,833.2900,2022,1,1,1,3
1,DIST,Demanda industrial en subtransmisión,Base,Variable (Energía),$/kWh,0.9899,2022,1,1,1,3
2,DIST,Demanda industrial en subtransmisión,Intermedia,Variable (Energía),$/kWh,1.6079,2022,1,1,1,3
3,DIST,Demanda industrial en subtransmisión,Punta,Variable (Energía),$/kWh,1.9042,2022,1,1,1,3
4,DIST,Demanda industrial en subtransmisión,Semipunta,Variable (Energía),$/kWh,0.0000,2022,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...
1,DIST,Demanda industrial en subtransmisión,Base,Variable (Energía),$/kWh,0.9445,2022,1,20,1477,23
2,DIST,Demanda industrial en subtransmisión,Intermedia,Variable (Energía),$/kWh,1.5862,2022,1,20,1477,23
3,DIST,Demanda industrial en subtransmisión,Punta,Variable (Energía),$/kWh,1.8298,2022,1,20,1477,23
4,DIST,Demanda industrial en subtransmisión,Semipunta,Variable (Energía),$/kWh,0.0000,2022,1,20,1477,23


In [7]:
driver.close()

In [61]:
dfs = []
driver = webdriver.Edge()
driver.implicitly_wait(10)
driver.get('https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/TarifasCREIndustria/Tarifas/DemandaIndustrialSub.aspx')
year_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha_ddAnio'))
year_selection.select_by_value('2022')
state_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddEstado'))
state_selection.select_by_value(str(1))

municip_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
municip_options = [option.text for option in municip_selection.options][1:]
municip_values = [option.get_attribute("value") for option in municip_selection.options][1:]

month_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha2_ddMes'))
month_selection.select_by_value(str(2))

print(municip_options)
print(municip_values)

for state, municip_index in zip(municip_options,municip_values):
    select_mun = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
    select_mun.select_by_value(str(municip_index))

    #select_div = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))
    select_div = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))
    div_options = [option.text for option in select_div.options][1:]
    div_values = [option.get_attribute("value") for option in select_div.options][1:]
    print(div_options)

    for division, division_index in zip(div_options,div_values):
        select_div.select_by_value(str(division_index))
        table = driver.find_element(By.CSS_SELECTOR,"table.table.table-bordered.table-striped")
        table_html = table.get_attribute('outerHTML')
        df = pd.read_html(table_html)[0]
        df['year'] = 2022
        df['month'] = month
        df['state'] = state
        df['municipality'] = municip_index
        df['division'] = division
        df['division_value'] = division_index
        dfs.append(df)
    

['AGUASCALIENTES', 'ASIENTOS', 'CALVILLO', 'COSIO', 'EL LLANO', 'JESUS MARIA', 'PABELLON DE ARTEAGA', 'RINCON DE ROMOS', 'SAN FRANCISCO DE LOS ROMO', 'SAN JOSE DE GRACIA', 'TEPEZALA']
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
['BAJÍO']
['BAJÍO']
['BAJÍO']
['BAJÍO']
['BAJÍO']
['BAJÍO']
['BAJÍO']
['BAJÍO']
['BAJÍO']
['BAJÍO']
['BAJÍO']


In [62]:
pd.concat(dfs, ignore_index=True)

,Tarifa,Descripción,Int. Horario,Cargo,Unidades,FEB-22,year,month,state,municipality,division,division_value
0,DIST,Demanda industrial en subtransmisión,-,Fijo,$/mes,833.2900,2022,1,AGUASCALIENTES,1,BAJÍO,3
1,DIST,Demanda industrial en subtransmisión,Base,Variable (Energía),$/kWh,0.9968,2022,1,AGUASCALIENTES,1,BAJÍO,3
2,DIST,Demanda industrial en subtransmisión,Intermedia,Variable (Energía),$/kWh,1.6201,2022,1,AGUASCALIENTES,1,BAJÍO,3
3,DIST,Demanda industrial en subtransmisión,Punta,Variable (Energía),$/kWh,1.9190,2022,1,AGUASCALIENTES,1,BAJÍO,3
4,DIST,Demanda industrial en subtransmisión,Semipunta,Variable (Energía),$/kWh,0.0000,2022,1,AGUASCALIENTES,1,BAJÍO,3
...,...,...,...,...,...,...,...,...,...,...,...,...
61,DIST,Demanda industrial en subtransmisión,Base,Variable (Energía),$/kWh,0.9968,2022,1,TEPEZALA,11,BAJÍO,3
62,DIST,Demanda industrial en subtransmisión,Intermedia,Variable (Energía),$/kWh,1.6201,2022,1,TEPEZALA,11,BAJÍO,3
63,DIST,Demanda industrial en subtransmisión,Punta,Variable (Energía),$/kWh,1.9190,2022,1,TEPEZALA,11,BAJÍO,3
64,DIST,Demanda industrial en subtransmisión,Semipunta,Variable (Energía),$/kWh,0.0000,2022,1,TEPEZALA,11,BAJÍO,3


In [8]:
dfs = []
driver = webdriver.Edge()
driver.implicitly_wait(10)
driver.get('https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/TarifasCRENegocio/Tarifas/GranDemandaMTH.aspx')
year_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha_ddAnio'))
year_selection.select_by_value('2022')
state_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddEstado'))
state_selection.select_by_value(str(12))

municip_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
municip_options = [option.text for option in municip_selection.options][1:]
municip_values = [option.get_attribute("value") for option in municip_selection.options][1:]

#print(municip_options)
#print(municip_values)
for month in range(1,11):
    month_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_Fecha2_ddMes'))
    month_selection.select_by_value(str(month))
    for state, municip_index in zip(municip_options,municip_values):
        select_mun = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddMunicipio'))
        select_mun.select_by_value(str(municip_index))

        #select_div = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))
        select_div = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))
        div_options = [option.text for option in select_div.options][1:]
        div_values = [option.get_attribute("value") for option in select_div.options][1:]
        #print(div_options)

        for division, division_index in zip(div_options,div_values):
            select_div = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddDivision'))
            select_div.select_by_value(str(division_index))
            try:
                table = driver.find_element(By.CSS_SELECTOR,"table.table.table-bordered.table-striped")
                table_html = table.get_attribute('outerHTML')
                df = pd.read_html(table_html)[0]
                df.columns = ['Tarifa','Descripción','Int. Horario','Cargo','Unidades','Valor']
                df['year'] = 2022
                df['month'] = month
                df['state'] = state
                df['municipality'] = municip_index
                df['division'] = division
                df['division_value'] = division_index
                dfs.append(df)
            except:
                pass
    

In [10]:
scraped_data = pd.concat(dfs, ignore_index=True)
scraped_data

,Tarifa,Descripción,Int. Horario,Cargo,Unidades,Valor,year,month,state,municipality,division,division_value
0,GDMTH,Gran demanda en media tensión horaria,-,Fijo,$/mes,314.3100,2022,1,ACAPULCO DE JUAREZ,366,CENTRO SUR,11
1,GDMTH,Gran demanda en media tensión horaria,Base,Variable (Energía),$/kWh,0.8747,2022,1,ACAPULCO DE JUAREZ,366,CENTRO SUR,11
2,GDMTH,Gran demanda en media tensión horaria,Intermedia,Variable (Energía),$/kWh,1.5321,2022,1,ACAPULCO DE JUAREZ,366,CENTRO SUR,11
3,GDMTH,Gran demanda en media tensión horaria,Punta,Variable (Energía),$/kWh,1.7319,2022,1,ACAPULCO DE JUAREZ,366,CENTRO SUR,11
4,GDMTH,Gran demanda en media tensión horaria,-,Distribución,$/kW,242.4300,2022,1,ACAPULCO DE JUAREZ,366,CENTRO SUR,11
...,...,...,...,...,...,...,...,...,...,...,...,...
5215,GDMTH,Gran demanda en media tensión horaria,Base,Variable (Energía),$/kWh,0.9263,2022,10,ZITLALA,446,CENTRO SUR,11
5216,GDMTH,Gran demanda en media tensión horaria,Intermedia,Variable (Energía),$/kWh,1.6331,2022,10,ZITLALA,446,CENTRO SUR,11
5217,GDMTH,Gran demanda en media tensión horaria,Punta,Variable (Energía),$/kWh,1.8479,2022,10,ZITLALA,446,CENTRO SUR,11
5218,GDMTH,Gran demanda en media tensión horaria,-,Distribución,$/kW,242.4300,2022,10,ZITLALA,446,CENTRO SUR,11


In [11]:
import os
def save_scraped_data(data,year='2022',state='aguascalientes'):
    state_path = './scraped_data/'+state
    if not os.path.exists(state_path):
        os.makedirs(state_path)
    data.to_csv(state_path+'/'+'scraped_data_'+year+'.csv',index=False)

In [12]:
save_scraped_data(scraped_data)

In [13]:
state_selection = Select(driver.find_element(By.ID,'ContentPlaceHolder1_EdoMpoDiv_ddEstado'))

In [17]:
estados = [option.text for option in state_selection.options][1:]

In [25]:
estados_dict = {i+1:estados[i] for i in range(len(estados))}

In [28]:
div_values = [option.get_attribute("value") for option in state_selection.options][1:]

In [29]:
div_values

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '9',
 '7',
 '8',
 '10',
 '15',
 '11',
 '12',
 '13',
 '14',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32']

In [5]:
import json
with open('estados_dict.json') as f:
        states = json.load(f)

In [6]:
states_keys = list(states.keys())
#states_keys = states_keys[:9]

In [7]:
states_keys[:9]

['1', '2', '3', '4', '5', '6', '9', '7', '8']

In [8]:
states_keys[9:]

['10',
 '15',
 '11',
 '12',
 '13',
 '14',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32']